**извлечение признаков из текста на естественном языке**

очистка текста и токенизация

_Евгений Борисов <esborisov@sevsu.ru>_

## библиотеки

In [1]:
import re
# import numpy as np
# import numpy.random as rng
import pandas as pd
# from tqdm import tqdm
from tqdm.notebook import tqdm

# np.set_printoptions(precision=2) # вывод на печать чисел до 2 знака
pd.options.display.max_colwidth = 200 

tqdm.pandas()

---

In [2]:
# https://habr.com/ru/post/516098/
# Corus — коллекция русскоязычных NLP-датасетов
# https://github.com/yutkin/Lenta.Ru-News-Dataset/

## тексты

In [3]:
# загружаем тексты
data = pd.read_pickle('../data/news.pkl.gz')
print('записей:',len(data))
data.sample(5)

записей: 3196


,text,tag
1362,Китайский космонавт рассказал о странном стуке в космосе\n\n30 ноября 2016 в 17:49\n\n42.TUT.BY\n\nКосмонавт Ян Ливэй в последнем интервью центральному китайскому телевидению рассказал о загадочно...,tech
1057,"Вырастут пенсии и цены на сигареты. Что меняется с 1 декабря\n\n30 ноября 2016 в 7:00\n\nFINANCE.TUT.BY\n\nУ пенсионеров появятся лишние примерно 14 рублей, по традиции подорожают сигареты, а забы...",economics
1111,"Первый в Беларуси ""Манекен челлендж"" с маленькими футболистами\n\n7 декабря 2016 в 11:16\n\nSPORT.TUT.BY\n\nПервый в Беларуси «Манекен челлендж» с маленькими футболистами сделал минский мини-футбо...",sport
2354,"По предварительным данным, пенсионерка и двое ее взрослых детей отравились газом\n\nВсе материалы сюжета Пожары, ДТП и криминальные события в Ростовской области\n\nОбстоятельства гибели трех челов...",incident
2660,"Ученые из Эдинбургского университета (Великобритания) допустили, что в атмосферах коричневых карликов — объектов, занимающих промежуточное положение между звездами и планетами, могут обитать живые...",science


In [4]:
# длина строк
pd.DataFrame(data['text'].str.len()).describe([.1,.25,.5,.75,.95]).astype(int).T

,count,mean,std,min,10%,25%,50%,75%,95%,max
text,3196,1720,2177,25,370,675,1070,1813,5761,30710


In [5]:
# количество категорий
data['tag'].drop_duplicates().count()

13

In [6]:
# собираем словарь из текстов
def get_vocabulary(ds):
    vcb = [ set(s) for s in ds.tolist() ]
    return sorted(set.union(*vcb))

## очистка и токенизация 

In [7]:
# применяет список замен pat к строке s
def replace_patterns(s,pat):
    if len(pat)<1: return s
    return  replace_patterns( re.sub(pat[0][0],pat[0][1],s), pat[1:] )

# нормализация текста
def string_normalizer(s):
    pat = [
       [r'ё','е'] # замена ё для унификации
       ,[r'</?[a-z]+>',' '] # удаляем xml
       ,[r'[^a-zа-я\- ]+',' '] # оставляем только буквы, пробел и -
       ,[r' -\w+',' '] # удаляем '-й','-тый' и т.п.
       ,[r'\w+- ',' ']
       ,[r' +',' '] # удаляем повторы пробелов
    ]
    return replace_patterns(s.lower(),pat).strip()

data['ctext'] = data['text'].progress_apply(string_normalizer)

  0%|          | 0/3196 [00:00<?, ?it/s]

In [8]:
# разрезаем стоки на слова
def tokenize(s): 
    return [ w for w in s.split(' ') if (len(w)>1) ]

data['ctext'] = data['ctext'].progress_apply( tokenize )

  0%|          | 0/3196 [00:00<?, ?it/s]

In [9]:
vcb =  get_vocabulary( data['ctext'] )
print('словарь %i слов'%(len(vcb)))
# pd.DataFrame( vcb ).to_csv('voc0.txt',index=False,header=False)

словарь 83094 слов


In [10]:
data.sample(3)

,text,tag,ctext
990,"Министр обороны Равков уверил Николая Статкевича в преданности армии белорусскому народу\n\n2 декабря 2016 в 15:08\n\nTUT.BY\n\nБелорусская армия готова защищать суверенитет, независимость, террит...",politics,"[министр, обороны, равков, уверил, николая, статкевича, преданности, армии, белорусскому, народу, декабря, tut, by, белорусская, армия, готова, защищать, суверенитет, независимость, территориальну..."
2772,"Корпорация Apple добавит второй слот для SIM-карты в новом iPhone. Новый гаджет будет представлен в конце 2017 года, передает Gizmochina со ссылкой на пост в китайской соцсети Weibo.\n\nСообщается...",tech,"[корпорация, apple, добавит, второй, слот, для, sim-карты, новом, iphone, новый, гаджет, будет, представлен, конце, года, передает, gizmochina, со, ссылкой, на, пост, китайской, соцсети, weibo, со..."
2014,"(обновлено 10:04 13.12.2016 )\n\nМОСКВА, 13 дек - Р-Спорт. Южнокорейские скелетонисты намерены бойкотировать чемпионат мира-2017 по бобслею и скелетону в Сочи из-за допингового скандала, сообщает ...",sport,"[обновлено, москва, дек, р-спорт, южнокорейские, скелетонисты, намерены, бойкотировать, чемпионат, по, бобслею, скелетону, сочи, из-за, допингового, скандала, сообщает, агентство, ренхап, со, ссыл..."


## стеминг

In [11]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords
# from nltk import download as nltk_download
# nltk_download('stopwords')

stopwords = set(nltk_stopwords.words('russian'))
stemmer = SnowballStemmer('russian')

# выкидываем stopwords, выполняем стеминг
def stem(s): 
    return [ stemmer.stem(w) for w in s if w not in stopwords ]

In [12]:
data['ctext'] = data['ctext'].progress_apply( stem )

  0%|          | 0/3196 [00:00<?, ?it/s]

In [13]:
vcb =  get_vocabulary( data['ctext'] )
print('словарь %i слов'%(len(vcb)))
# pd.DataFrame( vcb ).to_csv('voc1.txt',index=False,header=False)

словарь 36381 слов


In [14]:
data.sample(3)

,text,tag,ctext
1110,"Слуцкий после увольнения из ЦСКА отправится в Антарктиду\n\n7 декабря 2016 в 11:27\n\nСоветский спорт\n\nФото: Александр Федоров, «Спорт-Экспресс»\n\nКак стало известно «Советскому спорту», Леонид...",sport,"[слуцк, увольнен, цска, отправ, антарктид, декабр, советск, спорт, фот, александр, федор, спорт-экспресс, стал, известн, советск, спорт, леонид, слуцк, котор, матч, тоттенхэм, покинет, пост, главн..."
256,"Джордж Майкл последует примеру Элтона Джона Джордж Майкл сообщил, что он\nготов воспользоваться поправкой британского законодательства и\nзарегистрировать брак со своим давним партером Кенни Джосс...",culture,"[джордж, майкл, послед, пример, элтон, джон, джордж, майкл, сообщ, гот, воспользова, поправк, британск, законодательств, зарегистрирова, брак, сво, давн, партер, кен, джосс, напомн, декабр, сексуа..."
227,"Екатеринбурженка делает из кожи картины Ван Гога и Пикассо Жительница\nЕкатеринбурга Ираида Горячих из кусочков кожи делает репродукции картин\nизвестных художников, таких как Пабло Пикассо, Канди...",culture,"[екатеринбурженк, дела, кож, картин, ван, гог, пикасс, жительниц, екатеринбург, ираид, горяч, кусочк, кож, дела, репродукц, картин, известн, художник, так, пабл, пикасс, кандинск, винсент, ван, гог]"
